In [1]:
# Import SparkSession to create a Spark app 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Read inside Airbnb data").getOrCreate()

25/10/03 17:49:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
listing = spark.read.csv("Data/listings.csv.gz", header=True,
                        inferSchema=True, sep=",",
                        quote='"', escape='"',
                        multiLine=True, mode="PERMISSIVE")

In [3]:
for filed in listing.schema:
    print(filed)

StructField('id', LongType(), True)
StructField('listing_url', StringType(), True)
StructField('scrape_id', LongType(), True)
StructField('last_scraped', DateType(), True)
StructField('source', StringType(), True)
StructField('name', StringType(), True)
StructField('description', StringType(), True)
StructField('neighborhood_overview', StringType(), True)
StructField('picture_url', StringType(), True)
StructField('host_id', IntegerType(), True)
StructField('host_url', StringType(), True)
StructField('host_name', StringType(), True)
StructField('host_since', DateType(), True)
StructField('host_location', StringType(), True)
StructField('host_about', StringType(), True)
StructField('host_response_time', StringType(), True)
StructField('host_response_rate', StringType(), True)
StructField('host_acceptance_rate', StringType(), True)
StructField('host_is_superhost', StringType(), True)
StructField('host_thumbnail_url', StringType(), True)
StructField('host_picture_url', StringType(), True)


In [4]:
neighbourhoods = listing.select(listing.neighbourhood_cleansed)
neighbourhoods.show(20, truncate=False)

+----------------------+
|neighbourhood_cleansed|
+----------------------+
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Lewisham              |
|Camden                |
|Lambeth               |
|Richmond upon Thames  |
|Hillingdon            |
|Wandsworth            |
|Brent                 |
|Kensington and Chelsea|
|Greenwich             |
|Hackney               |
|Lambeth               |
|Hackney               |
+----------------------+
only showing top 20 rows


In [6]:
review_locations = listing.select(listing.review_scores_location)
review_locations.show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.62|
|                  4.54|
|                  4.36|
|                   4.5|
|                  4.37|
|                   4.5|
|                  4.53|
|                  4.52|
|                  4.49|
|                  4.94|
|                  4.82|
|                  4.92|
|                  4.14|
|                  4.78|
|                  4.43|
|                  4.73|
|                  NULL|
|                  NULL|
|                  4.48|
|                  4.75|
+----------------------+
only showing top 20 rows


In [7]:
listing.select(listing.review_scores_location).show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.62|
|                  4.54|
|                  4.36|
|                   4.5|
|                  4.37|
|                   4.5|
|                  4.53|
|                  4.52|
|                  4.49|
|                  4.94|
|                  4.82|
|                  4.92|
|                  4.14|
|                  4.78|
|                  4.43|
|                  4.73|
|                  NULL|
|                  NULL|
|                  4.48|
|                  4.75|
+----------------------+
only showing top 20 rows


In [43]:
listing     \
    .select(listing.review_scores_location)     \
    .show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.62|
|                  4.54|
|                  4.36|
|                   4.5|
|                  4.37|
|                   4.5|
|                  4.53|
|                  4.52|
|                  4.49|
|                  4.94|
|                  4.82|
|                  4.92|
|                  4.14|
|                  4.78|
|                  4.43|
|                  4.73|
|                  NULL|
|                  NULL|
|                  4.48|
|                  4.75|
+----------------------+
only showing top 20 rows


In [59]:
high_score_listing = listing.filter(listing.review_scores_location>4.5).select('id','price','name','review_scores_location')
high_score_listing.show(20,truncate=False)


+------+-------+------------------------------------------------+----------------------+
|id    |price  |name                                            |review_scores_location|
+------+-------+------------------------------------------------+----------------------+
|264776|$297.00|Huge Four Bedroom Apartment                     |4.62                  |
|264777|$98.00 |One Bedroom Apartment                           |4.54                  |
|264782|$120.00|One Bedroom Garden Apartment                    |4.53                  |
|264783|$216.00|Four Bedroom Garden Apartment                   |4.52                  |
|266037|$62.00 |Central London with Stunning Views!             |4.94                  |
|268398|$66.00 |Also five minutes to South Bank                 |4.82                  |
|270600|$73.00 |Patio Apartment in London (Twickenham)          |4.92                  |
|425143|NULL   |luxury 1bed in Chelsea Bridge Wharf             |4.78                  |
|426354|$200.00|1 bed

In [82]:
high_score_listing = listing \
    .filter(listing.review_scores_location>4.5) \
        .select('id','price','name','review_scores_location')

high_score_listing.dropna().show(20,truncate=False)

+------+-------+------------------------------------------------+----------------------+
|id    |price  |name                                            |review_scores_location|
+------+-------+------------------------------------------------+----------------------+
|264776|$297.00|Huge Four Bedroom Apartment                     |4.62                  |
|264777|$98.00 |One Bedroom Apartment                           |4.54                  |
|264782|$120.00|One Bedroom Garden Apartment                    |4.53                  |
|264783|$216.00|Four Bedroom Garden Apartment                   |4.52                  |
|266037|$62.00 |Central London with Stunning Views!             |4.94                  |
|268398|$66.00 |Also five minutes to South Bank                 |4.82                  |
|270600|$73.00 |Patio Apartment in London (Twickenham)          |4.92                  |
|426354|$200.00|1 bedroom flat with big balcony!                |4.73                  |
|427584|$129.00|Hackn

In [126]:
# Import functions we'll need for string replacement
from pyspark.sql.functions import regexp_replace, col


# Create a new DataFrame "df_no_dollar" by cleaning the "price" column
df_no_dollar = listing.withColumn(
    "price",                                   # Column we are updating
    regexp_replace("price", "[$,]", "")        # Remove "$" and "," characters from the string
    .cast('float')                             # Convert the cleaned string into a float type
        )

df_no_dollar.select('price','name').show(truncate=False)

+-----+--------------------------------------------------+
|price|name                                              |
+-----+--------------------------------------------------+
|297.0|Huge Four Bedroom Apartment                       |
|98.0 |One Bedroom Apartment                             |
|148.0|Two Bedroom Newly Refurbished Apartment           |
|144.0|Refurbished Two Bedroom Apartment                 |
|157.0|Spacious refurbished 2 bedroom apt with balcony   |
|148.0|Two Bedrooms Garden Maisonette                    |
|120.0|One Bedroom Garden Apartment                      |
|216.0|Four Bedroom Garden Apartment                     |
|238.0|Huge Three Bedroom Flat with parking and terrace  |
|62.0 |Central London with Stunning Views!               |
|66.0 |Also five minutes to South Bank                   |
|73.0 |Patio Apartment in London (Twickenham)            |
|NULL |Heathrow BNB - Home Away From Home!               |
|NULL |luxury 1bed in Chelsea Bridge Wharf              

In [128]:
df_no_dollar.schema['price']

StructField('price', FloatType(), True)

In [137]:
df_no_dollar.filter( (df_no_dollar.price<100) & (df_no_dollar.review_scores_rating>4.5) ) \
    .select('price','name','review_scores_rating')\
    .show(truncate=False)

+-----+-------------------------------------------------+--------------------+
|price|name                                             |review_scores_rating|
+-----+-------------------------------------------------+--------------------+
|98.0 |One Bedroom Apartment                            |4.58                |
|62.0 |Central London with Stunning Views!              |4.9                 |
|66.0 |Also five minutes to South Bank                  |4.63                |
|73.0 |Patio Apartment in London (Twickenham)           |4.64                |
|70.0 |Boutique Room w/ Private Bath, Balcony           |4.98                |
|72.0 |Holiday London DB Room Let-on going              |4.85                |
|74.0 |You are GUARANTEED to love this                  |4.87                |
|51.0 |Bright Double + workspace in spacious Garden Flat|4.75                |
|52.0 |SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT         |4.97                |
|75.0 |Room with a view, shared flat,  central  Bank

In [142]:
df_no_dollar.filter('price < 100 AND review_scores_rating > 4.5') \
    .select('price','name','review_scores_rating')\
    .show(truncate=False)

+-----+-------------------------------------------------+--------------------+
|price|name                                             |review_scores_rating|
+-----+-------------------------------------------------+--------------------+
|98.0 |One Bedroom Apartment                            |4.58                |
|62.0 |Central London with Stunning Views!              |4.9                 |
|66.0 |Also five minutes to South Bank                  |4.63                |
|73.0 |Patio Apartment in London (Twickenham)           |4.64                |
|70.0 |Boutique Room w/ Private Bath, Balcony           |4.98                |
|72.0 |Holiday London DB Room Let-on going              |4.85                |
|74.0 |You are GUARANTEED to love this                  |4.87                |
|51.0 |Bright Double + workspace in spacious Garden Flat|4.75                |
|52.0 |SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT         |4.97                |
|75.0 |Room with a view, shared flat,  central  Bank

In [151]:
listing.select(listing.property_type).distinct().show(50,truncate=False)


[Stage 53:>                                                         (0 + 1) / 1]

+----------------------------------+
|property_type                     |
+----------------------------------+
|Private room in lighthouse        |
|Private room in loft              |
|Private room in earthen home      |
|Entire chalet                     |
|Earthen home                      |
|Farm stay                         |
|Entire rental unit                |
|Shared room in hostel             |
|Shared room                       |
|Private room in condo             |
|Room in boutique hotel            |
|Private room in religious building|
|Room in bed and breakfast         |
|Private room in casa particular   |
|Private room in bungalow          |
|Entire cabin                      |
|Entire guesthouse                 |
|Hut                               |
|Private room in nature lodge      |
|Entire guest suite                |
|Private room in home              |
|Entire place                      |
|Floor                             |
|Camper/RV                         |
|

In [153]:
listing.select(listing.property_type, listing.room_type).distinct().show(50,truncate=False)


[Stage 59:>                                                         (0 + 1) / 1]

+----------------------------------+---------------+
|property_type                     |room_type      |
+----------------------------------+---------------+
|Room in hostel                    |Hotel room     |
|Private room in casa particular   |Private room   |
|Dome                              |Entire home/apt|
|Entire serviced apartment         |Entire home/apt|
|Private room in loft              |Private room   |
|Shipping container                |Entire home/apt|
|Private room in villa             |Private room   |
|Farm stay                         |Entire home/apt|
|Room in hotel                     |Hotel room     |
|Shared room in rental unit        |Shared room    |
|Private room in guest suite       |Private room   |
|Room in rental unit               |Hotel room     |
|Room in serviced apartment        |Hotel room     |
|Private room in serviced apartment|Private room   |
|Private room in hostel            |Private room   |
|Shared room                       |Shared roo

In [157]:
listing.select(listing.property_type).distinct().write.csv('Data/property_type/test.csv')